# Icsstrive.

## Importación de librerías.

In [143]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import country_converter as coco
import matplotlib.pyplot as plt
import re
import ast
from urllib.parse import urlparse

## Importación e información de dataframes.

In [144]:
df = pd.read_csv('./data/ICSSTRIVE.csv')
df

,data_source_link_url,description,date,locations,estimated_cost,victims,type_of_malware,threat_source,industries,impacts,references
0,https://icsstrive.com/incident/operational-imp...,ALPS' North American production operations and...,"September 10, 2023","['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...",No Malware identified,"[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']",OT IT,[{'title': 'Cyber Incident Victim: Alps Alpine...
1,https://icsstrive.com/incident/framework-lapto...,"San Francisco, California-based Framework, the...","January 9, 2024",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No Malware identified,No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...
2,https://icsstrive.com/incident/clothing-provid...,"After suffering a cyberattack December 13, V.F...","December 13, 2023",['United States'],"Cost not known, 35.5 million customers persona...","[{'title': 'V.F. Corporation', 'link_url': 'ht...",No Malware identified,No threat source identified,['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H..."
3,https://icsstrive.com/incident/hackers-accesse...,Toyota Financial Services (TFS) is warning cus...,"December 11, 2023",['Germany'],No cost values disclosed.,"[{'title': 'Toyota Financial Services, subs. o...",No Malware identified,No threat source identified,['Automotive'],IT Privacy,[{'title': 'Toyota warns customers of data bre...
4,https://icsstrive.com/incident/chinese-identif...,Chinese hackers are positioning themselves ins...,"December 11, 2023",['United States'],No cost values disclosed.,"[{'title': 'Unidentified Oil & Gas Pipeline', ...",No Malware identified,"[{'title': 'Volt Typhoon', 'link_url': 'https:...","['Oil & Gas', 'Water and Waste Water']",IT,[{'title': 'Chinese hackers allegedly target U...
...,...,...,...,...,...,...,...,...,...,...,...
1101,https://icsstrive.com/incident/ransomware-atta...,Bassett Furniture was forced to shut down its ...,"July 10, 2024",['United States'],"unknown, manufacturing shut down","[{'title': 'Bassett Furniture', 'link_url': 'h...",No Malware identified,No threat source identified,['Manufacturing'],OT IT,[{'title': 'Bassett Furniture reports signific...
1102,https://icsstrive.com/incident/rhysida-ransomw...,"The government of Columbus, Ohio shut down a l...","July 18, 2024",['United States'],$1.9M ransom demand; IT systems shut down,"[{'title': 'Ohio State Government', 'link_url'...",No Malware identified,"[{'title': 'Rhysida Ransomware Group', 'link_u...",['Government'],IT,[{'title': 'Columbus says it thwarted overseas...
1103,https://icsstrive.com/incident/threat-actor-ta...,"A threat actor known as ""Sticky Werewolf"" is u...","June 7, 2024","['Belarus', 'Russia']",No cost values disclosed.,"[{'title': 'Russian Aviation Industry', 'link_...","[{'title': 'Phishing Attack', 'link_url': 'htt...","[{'title': 'Sticky Werewolf', 'link_url': 'htt...","['Transportation (Includes Logistics, Shipping...",IT,[{'title': 'STICKY WEREWOLF TARGETS THE AVIATI...
1104,https://icsstrive.com/incident/wichita-city-ra...,"The cyberattack at the city of Wichita, Kansas...","May 3, 2024",['United States'],CIty services down for >1 month,"[{'title': 'City of Wichita', 'link_url': 'htt...",No Malware identified,"[{'title': 'LockBit', 'link_url': 'https://ics...","['Government', 'Water and Waste Water']",IT Privacy,[{'title': 'City of Wichita water bill payment...


In [145]:
df.describe(include='all')

,data_source_link_url,description,date,locations,estimated_cost,victims,type_of_malware,threat_source,industries,impacts,references
count,1106,1106,1106,1106,1106,1106,1106,1106,1106,723,1106
unique,1058,860,672,135,355,790,60,97,91,12,820
top,https://icsstrive.com/incident/mumu-infection-...,WIZARD SPIDER is a sophisticated eCrime group ...,"January 1, 2003",['United States'],No cost values disclosed.,No victims identified,No Malware identified,No threat source identified,"['Transportation (Includes Logistics, Shipping...",IT,No references identified.
freq,2,239,34,413,299,260,877,818,159,443,282


In [146]:
df.shape

(1106, 11)

In [147]:
df.dtypes

data_source_link_url    object
description             object
date                    object
locations               object
estimated_cost          object
victims                 object
type_of_malware         object
threat_source           object
industries              object
impacts                 object
references              object
dtype: object

In [148]:
df.columns

Index(['data_source_link_url', 'description', 'date', 'locations',
       'estimated_cost', 'victims', 'type_of_malware', 'threat_source',
       'industries', 'impacts', 'references'],
      dtype='object')

A continuación, se incluye una breve explicación sobre la información que aporta cada columna.
1. **Description**: Una breve descripción del incidente de ciberseguridad.
2. **Date**: La fecha en que se produjo o descubrió el incidente. Esto ayuda a comprender la cronología del ataque y analizar las tendencias.
3. **Locations**: Una lista de ubicaciones geográficas afectadas por el incidente, normalmente representadas por países o regiones.
4. **Estimated cost**: El impacto financiero estimado del ataque, que puede incluir pérdidas financieras directas, costes de recuperación o rescates pagados.
5. **Victims**: Detalles sobre las víctimas del ataque, normalmente organizaciones o empresas afectadas.
6. **Type of Malware**: El tipo de malware utilizado en el ataque, si se ha identificado.
7. **Threat source**: La fuente o actor detrás de la amenaza, como un grupo de hackers o un actor patrocinado por el Estado.
8. **Industries**: Las industrias afectadas por el ataque, como la automoción, la fabricación, el gobierno, etc. Esto ayuda a identificar los sectores objetivo del ataque.
9. **Impacts**: Los tipos de impactos del ataque, como la interrupción de las operaciones de TI, OT (tecnología operativa) o problemas de privacidad (exfiltración de datos).
10. **References**: Referencias adicionales al incidente, que pueden incluir enlaces a artículos de noticias o informes más detallados del incidente.
11. **Data source link url**: El enlace a la fuente primaria de información sobre el incidente, normalmente un informe o artículo en un sitio web relevante.


In [149]:
df.head(3)

,data_source_link_url,description,date,locations,estimated_cost,victims,type_of_malware,threat_source,industries,impacts,references
0,https://icsstrive.com/incident/operational-imp...,ALPS' North American production operations and...,"September 10, 2023","['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...",No Malware identified,"[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']",OT IT,[{'title': 'Cyber Incident Victim: Alps Alpine...
1,https://icsstrive.com/incident/framework-lapto...,"San Francisco, California-based Framework, the...","January 9, 2024",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No Malware identified,No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...
2,https://icsstrive.com/incident/clothing-provid...,"After suffering a cyberattack December 13, V.F...","December 13, 2023",['United States'],"Cost not known, 35.5 million customers persona...","[{'title': 'V.F. Corporation', 'link_url': 'ht...",No Malware identified,No threat source identified,['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H..."


Separaremos la columna **Date** en componentes de día, mes y año, cada uno de ellos como una columna diferente, convirtiéndola en un formato de fecha.

In [150]:
df['date'] = pd.to_datetime(df['date'])

In [151]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [152]:
df.drop('date', axis = 1, inplace = True)

In [153]:
df.head(2)

,data_source_link_url,description,locations,estimated_cost,victims,type_of_malware,threat_source,industries,impacts,references,year,month,day
0,https://icsstrive.com/incident/operational-imp...,ALPS' North American production operations and...,"['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...",No Malware identified,"[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']",OT IT,[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10
1,https://icsstrive.com/incident/framework-lapto...,"San Francisco, California-based Framework, the...",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No Malware identified,No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9


## Limpieza de datos.

### Manejo de filas duplicadas.
En caso de haber filas duplicadas, deberemos de eliminar estas para que no afecten al análisis posterior.

In [154]:
df.duplicated().sum() # Columnas duplicadas

48

In [155]:
df.drop_duplicates(inplace = True)

In [156]:
df.shape

(1058, 13)

### Manejo de filas nulas.

In [157]:
df.isnull().sum()

data_source_link_url      0
description               0
locations                 0
estimated_cost            0
victims                   0
type_of_malware           0
threat_source             0
industries                0
impacts                 336
references                0
year                      0
month                     0
day                       0
dtype: int64

Como estos valores nulos son pocos, los rellenaremos con la moda en este caso ya que es una variable de tipo objeto.

In [158]:
df['impacts'].fillna(df['impacts'].mode()[0], inplace=True)

In [159]:
df.shape[0]

1058

### Selección de variables categóricas y continuas.
Para realizar correctamente un ánalisis de datos debemos de distinguir entre las variables categóricas, variables que representan diferentes categorías o grupos, y continuas, variables que pueden tomar cualquier valor dentro de un rango determinado.

In [160]:
v_continuas = []
v_categoricas = []
for i in df.columns:
    if df[i].nunique() > 70 or df[i].dtypes in ['float64', 'int64']:
        v_continuas.append(i)
    else:
        v_categoricas.append(i)

print('Variables continuas: {}'.format(', '.join(v_continuas)))
print('Variables categóricas: {}'.format(', '.join(v_categoricas)))

Variables continuas: data_source_link_url, description, locations, estimated_cost, victims, threat_source, industries, references, year, month, day
Variables categóricas: type_of_malware, impacts


#### Tratamiento de variables categóricas.
Para el posterior modelo de clasificación, necesitamos codificar nuestras variables. Para ello, realizaremos una codificación ordinal que consiste en asignar valores enteros a las categorias basándose en su posición o jerarquía en los datos.

In [161]:
df[v_categoricas].head(3)

,type_of_malware,impacts
0,No Malware identified,OT IT
1,No Malware identified,IT
2,No Malware identified,IT


##### **Impact.**
El proceso consiste en agrupar los valores de la columna **Impacts** en categorías correspondientes. Como algunos valores pueden pertenecer a varias categorías, se realiza una asignación múltiple. Luego, se crean variables dummies para representar esas categorías, marcando con `1` las categorías presentes en cada fila. Esto facilita el análisis de datos en un formato adecuado para modelos y algoritmos.

In [162]:
df['impacts'].unique()

array(['OT IT', 'IT', 'IT Privacy', 'OT', 'Privacy', 'OT Privacy',
       'OT IT Privacy', ' IT', ' OT IT', ' IT Privacy', ' IT Safety',
       ' IT Safety Privacy'], dtype=object)

In [163]:
df['impacts'] = df['impacts'].str.strip() # Eliminar espacios en blanco.

In [164]:
impact_category_mapping = {
    "IT": ["IT", "IT Privacy", "IT Safety", "IT Safety Privacy", "OT IT"],
    "OT": ["OT", "OT Privacy", "OT IT"],
    "Privacy": ["Privacy", "OT Privacy", "IT Privacy"],
}

In [165]:
def map_categories(value, mapping):
    categories = []
    for category, terms in mapping.items():
        if any(term.strip() in value.split() for term in terms):
            categories.append(category)
    return categories

In [166]:
df['impacts'] = df['impacts'].apply(lambda x: ', '.join(set(map_categories(x, impact_category_mapping))))

In [167]:
df_impacts = df['impacts'].str.get_dummies(sep=', ')
df = pd.concat([df, df_impacts], axis=1)

In [168]:
df.head(4)

,data_source_link_url,description,locations,estimated_cost,victims,type_of_malware,threat_source,industries,impacts,references,year,month,day,IT,OT,Privacy
0,https://icsstrive.com/incident/operational-imp...,ALPS' North American production operations and...,"['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...",No Malware identified,"[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']","IT, OT",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10,1,1,0
1,https://icsstrive.com/incident/framework-lapto...,"San Francisco, California-based Framework, the...",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No Malware identified,No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9,1,0,0
2,https://icsstrive.com/incident/clothing-provid...,"After suffering a cyberattack December 13, V.F...",['United States'],"Cost not known, 35.5 million customers persona...","[{'title': 'V.F. Corporation', 'link_url': 'ht...",No Malware identified,No threat source identified,['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H...",2023,12,13,1,0,0
3,https://icsstrive.com/incident/hackers-accesse...,Toyota Financial Services (TFS) is warning cus...,['Germany'],No cost values disclosed.,"[{'title': 'Toyota Financial Services, subs. o...",No Malware identified,No threat source identified,['Automotive'],"IT, Privacy",[{'title': 'Toyota warns customers of data bre...,2023,12,11,1,0,1


##### **Type of Malware.**
El proceso consiste en convertir las cadenas de la columna, que pueden representar listas de diccionarios, en estructuras adecuadas para extraer la información relevante.

Primero, las cadenas se convierten en listas de diccionarios, manejando posibles errores de formato mediante excepciones para evitar fallos. Cuando la conversión es exitosa, se extraen los valores de las claves `title` y `link_url` de los diccionarios dentro de la lista, asignando valores predeterminados como `No Malware Detected` si no se encuentran datos. Esto permite crear dos nuevas columnas: **Malware title** y **Malware URL**.

In [169]:
df['type_of_malware'].unique()

array(['No Malware identified',
       "[{'title': 'LockBit', 'link_url': 'https://icsstrive.com/malware/lockbit/'}]",
       "[{'title': 'Unknown', 'link_url': 'https://icsstrive.com/malware/unknown-2/'}]",
       "[{'title': 'DDoS Attack', 'link_url': 'https://icsstrive.com/malware/ddos-attack/'}]",
       "[{'title': 'Unknown ransomware variant', 'link_url': 'https://icsstrive.com/malware/unknown-ransomware-variant/'}]",
       "[{'title': 'Ransomware – unknown', 'link_url': 'https://icsstrive.com/malware/ransomware-4/'}]",
       "[{'title': 'Unknown phishing attack', 'link_url': 'https://icsstrive.com/malware/unknown-phishing-attack/'}]",
       "[{'title': 'Ransomware – Unknown group or variant', 'link_url': 'https://icsstrive.com/malware/ransomware-3/'}]",
       "[{'title': 'LightlessCan malware', 'link_url': 'https://icsstrive.com/malware/lightlesscan-malware/'}]",
       "[{'title': 'BlackCat', 'link_url': 'https://icsstrive.com/malware/blackcat/'}]",
       "[{'title': 'Phis

In [170]:
df[df['type_of_malware'] == 'No Malware identified'].shape

(830, 16)

In [171]:
def safely_parse(value):
    try:
        return ast.literal_eval(value) if isinstance(value, str) else value
    except:
        return []

In [172]:
df['type_of_malware'] = df['type_of_malware'].apply(safely_parse)

In [173]:
df['malware_title'] = df['type_of_malware'].apply(lambda x: x[0]['title'] if isinstance(x, list) and x else 'No Malware identified')
df['malware_url'] = df['type_of_malware'].apply(lambda x: x[0]['link_url'] if isinstance(x, list) and x else 'No Malware identified')

In [174]:
df.drop('type_of_malware', axis = 1, inplace = True)

**Malware URL.**

Deberemos de extraer el nombre del dominio de la URL de la nueva columna **Malware URL**.

In [175]:
def extract_domain(url):
    try:
        if not url or "http" not in url:
            return "N/A"
        return urlparse(url).netloc
    except:
        return "N/A"

df['malware_url'] = df['malware_url'].apply(extract_domain)

In [176]:
df['malware_url'].unique()

array(['N/A', 'icsstrive.com'], dtype=object)

**Malware Title.**

In [177]:
df['malware_title'].unique()

array(['No Malware identified', 'LockBit', 'Unknown', 'DDoS Attack',
       'Unknown ransomware variant', 'Ransomware – unknown',
       'Unknown phishing attack', 'Ransomware – Unknown group or variant',
       'LightlessCan malware', 'BlackCat', 'Phishing Attack', 'CLOP',
       'Agenda', 'Ragnar Locker', 'Quantum Ransomware', 'LV', 'PLAY',
       'RansomEXX', '0Ktapus phishing campaign', 'ROADSWEEP',
       'Ransomware [Target: AU Prison]', 'Flame', 'Conti Ransomware',
       'Bitlocker Ransomware', 'NotPetya', 'BlackByte',
       'Ranwomare attack', 'Maui ransomware', 'Snatch Ransomware',
       'Colossus', 'BlackMatter', 'Meteor', 'REvil', 'Avaddon',
       'DarkSide', 'Related to Accellion File Transfer Appliance (FTA)',
       'Phoenix Locker', 'DoppelPaymer', 'Ransomware (target: Gyrodata)',
       'Nefilim Ransomware', 'Ryuk ransomware', 'Netfilim', 'Egregor',
       'Unknown malware attack', 'Mailto', 'Matriex', 'Winnti', 'EKANS',
       'SNAKE', 'unknown ransomware, but unse

In [178]:
malware_mapping = {
    "Ransomware": ["Ransomware"],
    "Spyware": ["Spyware"],
    "Trojan": ["Trojan"],
    "Worm": ["Worm"],
    "No Malware identified": ["No Malware identified", "Unknown malware attack",],
}

**SEGUIR**
**************************************************************

### Tratamiento de variables continuas.
Para el modelo de clasificación debemos de pasar todas las columnas a tipo `int`. Por ello, debemos de ver si realizar como en el caso anterior una codificación ordinal o agruparlos.

In [179]:
df[v_continuas].head(2)

,data_source_link_url,description,locations,estimated_cost,victims,threat_source,industries,references,year,month,day
0,https://icsstrive.com/incident/operational-imp...,ALPS' North American production operations and...,"['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...","[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10
1,https://icsstrive.com/incident/framework-lapto...,"San Francisco, California-based Framework, the...",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No threat source identified,['Technology'],[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9


In [180]:
v_categoricas_cont = list(df[v_continuas].dtypes[df[v_continuas].dtypes == 'object'].index)

In [181]:
df[v_categoricas_cont].nunique()

data_source_link_url    1058
description              860
locations                135
estimated_cost           355
victims                  790
threat_source             97
industries                91
references               820
dtype: int64

#### **Data Source Link URL.**
Deberemos de extraer el nombre del dominio de la URL.

In [182]:
def extract_domain(url):
    try:
        return urlparse(url).netloc
    except:
        return None

df['data_source_link_url'] = df['data_source_link_url'].apply(extract_domain)

In [183]:
df['data_source_link_url'] = df['data_source_link_url'].str.replace('www.', '', regex=False)

In [184]:
df.head(3)

,data_source_link_url,description,locations,estimated_cost,victims,threat_source,industries,impacts,references,year,month,day,IT,OT,Privacy,malware_title,malware_url
0,icsstrive.com,ALPS' North American production operations and...,"['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...","[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']","IT, OT",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10,1,1,0,No Malware identified,N/A
1,icsstrive.com,"San Francisco, California-based Framework, the...",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9,1,0,0,No Malware identified,N/A
2,icsstrive.com,"After suffering a cyberattack December 13, V.F...",['United States'],"Cost not known, 35.5 million customers persona...","[{'title': 'V.F. Corporation', 'link_url': 'ht...",No threat source identified,['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H...",2023,12,13,1,0,0,No Malware identified,N/A


#### **Locations.**
Para realizar una evaluación más efectiva de los países en nuestros datos, es conveniente agruparlos por continentes. Esto es especialmente relevante porque en otros archivos CSV hay registros de ataques provenientes de diferentes países, distribuidos en distintos continentes.

Para facilitar este análisis, procederemos a crear seis nuevas columnas en nuestro conjunto de datos, cada una representando un continente específico. Estas columnas indicarán con un valor de 1 si el país pertenece al continente correspondiente y 0 si no es así, es decir, mediante una asignación de valores binarios.

In [185]:
df['locations'].unique()

array(["['Japan', 'United States']", "['United States']", "['Germany']",
       "['Trinidad & Tobago']", "['Curaçao']",
       "['Australia', 'New Zealand']", "['Australia', 'United States']",
       "['Italy', 'United Kingdom']", "['France']", "['Japan']",
       "['Netherlands', 'United States']", "['United Kingdom']",
       "['Australia']", "['Germany', 'United States']", "['India']",
       "['Russia']", "['Philippines']", "['Canada']", "['Taiwan']",
       "['Spain']", "['Italy']", "['Bermuda']", "['Israel']",
       "['New Zealand']", "['France', 'Netherlands']",
       "['Canada', 'United States']", "['Sri Lanka']",
       "['Germany', 'Netherlands']", "['Poland']", "['Denmark']",
       "['Portugal']", "['China']", "['Kenya']", "['Norway']",
       "['Sweden', 'United Kingdom']", "['Ireland']",
       "['Germany', 'Global']", "['Belgium', 'Netherlands']",
       "['Netherlands']", "['Switzerland']", '[]', "['Belgium']",
       "['France', 'Global']", "['Sweden']", "['Croatia']

Vemos que existen los caracteres `[` y `]`, así como las comillas, y los eliminaremos para dejar las cadenas de texto más limpias y legibles.

In [186]:
df['locations'] = df['locations'].str.strip("[]").str.replace("'", "")

In [187]:
def assign_continent(country):
    try:
        return coco.convert(names=country, to='continent')
    except:
        return None # Si no se puede asignar un continente, se asigna un valor nulo.

In [188]:
def assign_continents_to_countries(countries):
    country_list = countries.split(', ')
    # Usar un conjunto para evitar duplicados y hacer la búsqueda más eficiente
    continents_set = set()
    for country in country_list:
        continent = assign_continent(country)
        if continent and continent not in continents_set:
            continents_set.add(continent)
        elif continent and continent in countries:
            continue
    return ', '.join(continents_set)



In [189]:
df['continents'] = df['locations'].apply(assign_continents_to_countries)

Global not found in regex
 not found in regex
Global not found in regex
Global not found in regex
 not found in regex
 not found in regex
 not found in regex
 not found in regex
Asia-Pacific not found in regex
 not found in regex
Global not found in regex
 not found in regex
Global not found in regex
 not found in regex
 not found in regex
 not found in regex
 not found in regex
Europe not found in regex
South America not found in regex
Africa not found in regex
Europe not found in regex
Middle East not found in regex
Global not found in regex
Global not found in regex
Asia-Pacific not found in regex
Europe not found in regex
Global not found in regex
Global not found in regex
Global not found in regex
Global not found in regex
Global not found in regex
Global not found in regex
Europe not found in regex
Global not found in regex
Asia-Pacific not found in regex
Global not found in regex
Middle East not found in regex
Global not found in regex
Global not found in regex
Europe not found 

In [ ]:
# continent_dummies = df['continents'].str.get_dummies(sep=', ')

In [18]:
# Función para limpiar la columna 'estimated_cost'
def clean_estimated_cost(cost_string):
    if pd.isna(cost_string):
        return "N/A"

    # Asegurarse de que el valor sea un string
    cost_string = str(cost_string).strip()

    # Filtrar valores conocidos no válidos
    if any(word in cost_string.lower() for word in ['no cost values disclosed', 'unknown', 'unsure']):
        return "N/A"

    if re.search(r'\d+\+?\s*(days|hours|minutes|seconds|weeks)', cost_string.lower()):
        return "N/A"

    # Detectar valores con "$<número><M o K>"
    match = re.search(r'\$\s*(\d+(?:[.,]\d{1,2})?)(\s?[KkMm]?)', cost_string)
    if match:
        number = match.group(1).replace(',', '')
        unit = match.group(2).upper().strip()

        try:
            number = float(number)
        except ValueError:
            return "N/A"

        if unit == 'M':
            return number * 1_000_000
        elif unit == 'K':
            return number * 1_000
        else:
            return number

    # Detectar rangos, por ejemplo: $18 - $20 million
    match_range = re.search(r'\$\s*(\d+(?:[.,]\d{1,2})?)\s*[-to]+\s*(\d+(?:[.,]\d{1,2})?)\s*(million|M|K|k)?', cost_string)
    if match_range:
        low_value = match_range.group(1).replace(',', '')
        high_value = match_range.group(2).replace(',', '')
        unit = match_range.group(3).upper() if match_range.group(3) else None

        try:
            low_value = float(low_value)
            high_value = float(high_value)
        except ValueError:
            return "N/A"

        if unit in ['M', 'million']:
            low_value *= 1_000_000
            high_value *= 1_000_000
        elif unit in ['K', 'k']:
            low_value *= 1_000
            high_value *= 1_000

        return f"{int(low_value)}-{int(high_value)}"

    # Si no se encuentra ningún valor numérico válido, retornar N/A
    return "N/A"

# Categorizar valores de costo estimado
def categorize_estimated_cost(value):
    if value == "N/A" or pd.isna(value):
        return "Unknown"

    try:
        if isinstance(value, float):
            cost_value = value
        elif isinstance(value, str) and '-' in value:
            low, high = map(float, value.split('-'))
            cost_value = (low + high) / 2
        else:
            cost_value = float(value)
    except ValueError:
        return "Unknown"

    if cost_value < 100_000:
        return "Low"
    elif 100_000 <= cost_value < 1_000_000:
        return "Medium"
    elif 1_000_000 <= cost_value < 10_000_000:
        return "High"
    else:
        return "Very High"

# Cargar el archivo CSV (reemplaza con la ruta correcta si es necesario)
df = pd.read_csv('ICSSTRIVE.csv')  # Cargar el archivo CSV

# Comprobar si la columna 'estimated_cost' existe en el DataFrame
if 'estimated_cost' in df.columns:
    # Limpiar la columna 'estimated_cost'
    df['estimated_cost'] = df['estimated_cost'].apply(clean_estimated_cost)
    print("\nColumna 'estimated_cost' modificada:")
    print(df[['estimated_cost']].head(67))

    # Aplicar la categorización a la columna 'estimated_cost'
    df['cost_category'] = df['estimated_cost'].apply(categorize_estimated_cost)
    print("\nColumna 'cost_category' modificada:")
    print(df[['estimated_cost', 'cost_category']].head(67))
else:
    print("La columna 'estimated_cost' no existe en el DataFrame.")



Columna 'estimated_cost' modificada:
   estimated_cost
0             N/A
1             N/A
2             N/A
3             N/A
4             N/A
..            ...
62            N/A
63            N/A
64            N/A
65      1500000.0
66            N/A

[67 rows x 1 columns]

Columna 'cost_category' modificada:
   estimated_cost cost_category
0             N/A       Unknown
1             N/A       Unknown
2             N/A       Unknown
3             N/A       Unknown
4             N/A       Unknown
..            ...           ...
62            N/A       Unknown
63            N/A       Unknown
64            N/A       Unknown
65      1500000.0          High
66            N/A       Unknown

[67 rows x 2 columns]


In [19]:
import pandas as pd

# Función para limpiar la columna 'victims' y categorizar
def clean_victims(victim_string):
    if pd.isna(victim_string):
        return "N/A"

    try:
        # Convertir la cadena en una lista de diccionarios
        victim_list = eval(victim_string)
        victim_names = [victim['title'] for victim in victim_list if 'title' in victim]

        # Categorizar las víctimas según patrones
        categorized_victims = categorize_victims(victim_names)
        return ', '.join(categorized_victims) if categorized_victims else "N/A"
    except Exception:
        return "N/A"

# Función para categorizar las víctimas según patrones comunes
def categorize_victims(victim_names):
    categories = []

    for name in victim_names:
        if 'Group' in name or 'Inc.' in name:
            categories.append('Corporation/Group')
        elif 'Corporation' in name:
            categories.append('Corporation')
        elif 'Company' in name:
            categories.append('Company')
        else:
            categories.append('Other')

    return list(set(categories))


# Cargar el archivo CSV (reemplaza con la ruta correcta si es necesario)
df = pd.read_csv('ICSSTRIVE.csv')  # Cargar el archivo CSV

# Verificar si la columna 'victims' existe en el DataFrame antes de procesar
if 'victims' in df.columns:
    # Limpiar y categorizar la columna 'victims'
    df['victims'] = df['victims'].apply(clean_victims)
    print("\nColumna 'victims' modificada:")
    print(df[['victims']].head(20))
else:
    print("La columna 'victims' no existe en el DataFrame.")



Columna 'victims' modificada:
              victims
0   Corporation/Group
1               Other
2         Corporation
3               Other
4               Other
5               Other
6               Other
7               Other
8               Other
9               Other
10              Other
11  Corporation/Group
12              Other
13              Other
14            Company
15              Other
16              Other
17              Other
18              Other
19  Corporation/Group


In [ ]:
# Función para limpiar la columna 'threat_source'
def clean_threat_source(threat_source_string):
    if pd.isna(threat_source_string):
        return "N/A", "N/A"  # Devuelve "N/A" si es un valor nulo

    if "No threat source identified" in threat_source_string:
        return "No threat source identified", "N/A"

    try:
        # Evaluar el string para convertirlo en una lista de diccionarios
        threat_list = eval(threat_source_string)
        threat_titles = []
        threat_links = []

        # Iterar sobre los elementos de la lista
        for threat in threat_list:
            if 'title' in threat:
                title = threat['title']

                # Verificar si contiene "unknown" (mayúsculas o minúsculas)
                if 'unknown' in title.lower():
                    title = 'Unknown'  # Reemplazar por "Unknown"

                threat_titles.append(title)

            # Extraer el link URL del threat source si existe
            if 'link_url' in threat:
                link_url = threat['link_url']
                domain = extract_domain(link_url)
                threat_links.append(domain)

        return (
            ', '.join(threat_titles) if threat_titles else "N/A",
            ', '.join(threat_links) if threat_links else "N/A"
        )

    except Exception:
        return "N/A", "N/A"  # Devuelve "N/A" si ocurre algún error

# Función para extraer el dominio de una URL
def extract_domain(url):
    try:
        parsed_url = urlparse(url)
        return parsed_url.netloc
    except Exception:
        return "N/A"  # Devuelve "N/A" si ocurre algún error


# Limpiar la columna 'threat_source' y crear nuevas columnas
if 'threat_source' in df.columns:
    df[['Threat source title', 'Threat source link URL']] = df['threat_source'].apply(
        lambda x: pd.Series(clean_threat_source(x))
    )
else:
    print("La columna 'threat_source' no existe en el DataFrame.")
    exit()

# Codificar las categorías de 'Threat source title' usando LabelEncoder
label_encoder = LabelEncoder()
df['Threat source title Encoded'] = label_encoder.fit_transform(df['Threat source title'].fillna('Unknown'))

# Ver las primeras filas del DataFrame con las nuevas columnas
print("\nPrimeras filas del DataFrame con nuevas columnas:")
print(df[['Threat source title', 'Threat source link URL', 'Threat source title Encoded']].head(20))



Primeras filas del DataFrame con nuevas columnas:
            Threat source title Threat source link URL  \
0                     BlackByte          icsstrive.com   
1   No threat source identified                    N/A   
2   No threat source identified                    N/A   
3   No threat source identified                    N/A   
4                  Volt Typhoon          icsstrive.com   
5                       LockBit          icsstrive.com   
6                Cyber Av3ngers          icsstrive.com   
7         Akira Ransomware Gang          icsstrive.com   
8   No threat source identified                    N/A   
9         Hunters International          icsstrive.com   
10               Cyber Av3ngers          icsstrive.com   
11        Akira Ransomware Gang          icsstrive.com   
12               Cyber Av3ngers          icsstrive.com   
13  No threat source identified                    N/A   
14          Play, aka PlayCrypt          icsstrive.com   
15  No threat source 

In [22]:
# Función para limpiar la columna 'industries'
def clean_industries(industries_string):
    if pd.isna(industries_string) or industries_string == "[]":
        return "N/A"  # Si está vacío o es una lista vacía, devolver "N/A"

    try:
        # Evaluar el string para convertirlo en una lista
        industries_list = eval(industries_string)

        # Si la lista tiene elementos, devolver el primer elemento o todos
        if industries_list:
            return ', '.join(industries_list)
        else:
            return "N/A"
    except Exception:
        return "N/A"  # Si ocurre un error, devolver "N/A"

# Asignación de categorías a números (Label Encoding)
category_mapping = {
    "Manufacturing": 0,
    "Technology": 1,
    "Healthcare": 2,
    "Transportation and Logistics": 3,
    "Energy and Utilities": 4,
    "Government and Defense": 5,
    "Business Services": 6,
    "Public Sector and Non-Profit": 7,
    "Finance": 8
}

# Función para agrupar las industrias en categorías
def group_industries(industries_string):
    # Listas de industrias asociadas a cada categoría
    manufacturing = ["Aerospace", "Automotive", "Chemicals", "Metals and Mining", "Steel", "Food and Beverage", "Pulp and Paper", "Manufacturing"]
    technology = ["Software", "Cloud Provider", "Communications", "Technology"]
    healthcare = ["Biotech", "Pharmaceutical", "Healthcare"]
    transportation = ["Transportation (Includes Logistics, Shipping, Maritime, Rail, Trucking"]
    energy = ["Oil and Gas", "Water and Waste Water"]
    government_defense = ["Government", "Defense"]
    business_services = ["Business"]
    public_sector = ["Other"]
    finance = ["Banking", "Insurance"]

    # Convertir la cadena a lista
    industries_list = industries_string.split(', ')

    # Asignar categorías basadas en la industria
    if any(industry in industries_list for industry in manufacturing):
        return "Manufacturing"
    elif any(industry in industries_list for industry in technology):
        return "Technology"
    elif any(industry in industries_list for industry in healthcare):
        return "Healthcare"
    elif any(industry in industries_list for industry in transportation):
        return "Transportation and Logistics"
    elif any(industry in industries_list for industry in energy):
        return "Energy and Utilities"
    elif any(industry in industries_list for industry in government_defense):
        return "Government and Defense"
    elif any(industry in industries_list for industry in business_services):
        return "Business Services"
    elif any(industry in industries_list for industry in public_sector):
        return "Public Sector and Non-Profit"
    elif any(industry in industries_list for industry in finance):
        return "Finance"
    else:
        return "Other"  # Para casos no definidos


# Limpiar y procesar la columna 'industries'
if 'industries' in df.columns:
    # Limpiar la columna 'industries'
    df['industries'] = df['industries'].apply(clean_industries)

    # Agrupar las industrias en categorías
    df['industries_grouped'] = df['industries'].apply(group_industries)

    # Aplicar Label Encoding a 'industries_grouped'
    le = LabelEncoder()
    df['industries_encoded'] = le.fit_transform(df['industries_grouped'])

    # Mostrar las primeras filas del DataFrame procesado
    print("\nDataFrame con columnas relacionadas a 'industries':")
    print(df[['industries', 'industries_grouped', 'industries_encoded']].head(20))
else:
    print("La columna 'industries' no existe en el DataFrame.")



DataFrame con columnas relacionadas a 'industries':
                                           industries      industries_grouped  \
0                           Automotive, Manufacturing           Manufacturing   
1                                          Technology              Technology   
2                                       Manufacturing           Manufacturing   
3                                          Automotive           Manufacturing   
4                    Oil & Gas, Water and Waste Water    Energy and Utilities   
5                               Water and Waste Water    Energy and Utilities   
6                               Water and Waste Water    Energy and Utilities   
7   Energy (Includes Power and Utilities), Water a...    Energy and Utilities   
8                                   Automotive, Other           Manufacturing   
9   Transportation (Includes Logistics, Shipping, ...                   Other   
10                              Water and Waste Water   